## 0. Import packages

In [1]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
from Association import Association
import numpy as np
tqdm.pandas() #for progres_apply etc.

## 1. Load data from previous step

In [2]:
import pickle
rules = pickle.load(open("pickle_dumps/article_rules.p", "rb"))
user_rules = pickle.load(open("pickle_dumps/user_rules.p", "rb"))

df_test = pickle.load(open("pickle_dumps/test_df.p", "rb"))
df_test = df_test.set_index("user_id").to_dict()["item_id"]

df_train = pickle.load(open("pickle_dumps/train_df.p", "rb"))
df_train = df_train.set_index("user_id").to_dict()["item_id"]

In [3]:
print(len(rules))
for i in range(10):
    if i < len(rules):
        print(rules[i])

3823503
Conf: 1.00	Supp: 55.00	 {512, 4, 1576, 778, 2672, 1328, 2353, 1555} => {2970}
Conf: 1.00	Supp: 55.00	 {512, 4, 1576, 778, 2672, 1328, 2353, 2970} => {1555}
Conf: 1.00	Supp: 54.00	 {512, 481, 736, 4, 1576, 1328, 2353, 1555} => {2970}
Conf: 1.00	Supp: 54.00	 {512, 4, 1576, 2672, 785, 1328, 2353, 1555} => {2970}
Conf: 1.00	Supp: 54.00	 {512, 4, 1576, 1839, 2672, 2353, 1328, 1555} => {2970}
Conf: 1.00	Supp: 54.00	 {512, 1576, 1839, 2672, 2353, 1328, 1555, 2970} => {4}
Conf: 1.00	Supp: 53.00	 {512, 1576, 2672, 785, 2258, 2353, 1555, 1328} => {2970}
Conf: 1.00	Supp: 53.00	 {512, 1576, 2672, 785, 2258, 2353, 1555, 2970} => {1328}
Conf: 1.00	Supp: 53.00	 {512, 4, 1576, 1328, 2353, 2258, 785, 1555} => {2970}
Conf: 1.00	Supp: 53.00	 {512, 4, 1576, 2353, 2258, 785, 1555, 2970} => {1328}


In [4]:
class Recommender():
    def __init__(self, assos):
        pass
    
    def recommend(self, user_items, k, user_id=None, split=None):
        pass        

In [5]:
def splitUser(user_items):
    splits = []
    
#     scores = cross_val_score(clf, X, y, cv=5)
    tmp = np.array_split(user_items, 4)
    
    for l in tmp:
        if len(l) > 0:
            splits.append(list(l))
    
    res = []
    for i in splits:
        join = []
        for j in splits:
            if i != j:
                join += j
        res.append((join, i))
                    
                

    return res

print(splitUser(range(10)))

[([3, 4, 5, 6, 7, 8, 9], [0, 1, 2]), ([0, 1, 2, 6, 7, 8, 9], [3, 4, 5]), ([0, 1, 2, 3, 4, 5, 8, 9], [6, 7]), ([0, 1, 2, 3, 4, 5, 6, 7], [8, 9])]


In [20]:
def hitrate(rc, k):
    hits = 0
    ndcg = []
    counts = 0
    avg_len = 0
    
    idcg = sum(1/np.log2(i+1) for i in range(1,k+1))
    
    for user in tqdm(df_test):
        user_items = df_test[user]
        split_user = splitUser(user_items)
        for split_id, split in enumerate(split_user):
            r = rc.recommend(split[0], k, user, split_id)
        
            avg_len += len(r)/len(split_user)
            
            for item in split[1]:
                if item in r:
                    hits += (item in r) / len(split_user)
                    break
            
            dcg = 0
            for i, rec in enumerate(r):
                good = 1 if rec in split[1] else 0
                dcg += (2**good - 1)/(np.log2(i+2))
            ndcg.append(dcg/idcg)
            
                
                
        counts += 1
    
    print("avg len: {:f}".format(avg_len / len(df_test)))
    if k == -1:
        print("HR\t{:.5f}".format((hits / counts)))
    else:
        print("HR@{}\t{:.5f}".format(k, (hits / counts)))
    print("nDCG@{}\t{:.5f}".format(k, (sum(ndcg) / len(ndcg))))
    return hits / counts

In [7]:
class Assos_Recommender(Recommender):
    def __init__(self, assos):
        # use popular items to fill unused space
        # TODO (df_train is used for this)
        
        self.assos = sorted(assos, key=lambda x: x.s * x.c, reverse=True)
        
#     def get_items(self, user_id):
#         return self.df_test[user_id]
    
    def recommend(self, user_items, k, user_id=None, split=None):
        recommendation = set()
        for a in self.assos:
            for l in a.left:
                # break if any element is not in the user session
                if l not in user_items:
                    break
                    
                # all elements are in the user session --> add RHS of rule to the recommendation
                r = list(a.right)[0]
                if r not in user_items:
                    recommendation.add(r)
                        
            if len(recommendation) >= k and k != -1:
                break
                
        return recommendation

# Results
rule.s --> 18.317 <br>
rule.s * rule.c --> 20.545

In [8]:
ar = Assos_Recommender(rules)
print("go")
hr = hitrate(ar, 10)

go


100%|██████████| 101/101 [02:35<00:00,  1.54s/it]

avg len: 10.000000
HR@10	0.20050


In [9]:
class User_Recommender(Recommender):
    def __init__(self, assos):
        self.assos = assos
    
    def recommend(self, user_items, k, user_id, split):
        user_items = set(user_items)
        rdict = dict()
        for a in self.assos[split]:
            if user_id in a.right:
                item_list = list()
                for l in a.left:
                    if l in df_test:
                        item_list.append(set(df_test[l]))
                    if l in df_train:
                        item_list.append(set(df_train[l]))
                        
                if item_list:
                    intersection = set.intersection(*item_list)
                    for item in intersection - user_items:
                        if item in rdict:
                            rdict[item] += (a.c * a.s)
                        else:
                            rdict[item] = (a.c * a.s)

                            
#         if rdict.values():   
#             if max(rdict.values()) < 50:
#                 return set()
            
        rlist = sorted(rdict.keys(), key=lambda x: rdict[x], reverse=True)

        recommendation = set()
        for i in rlist:
            recommendation.add(i)
            if len(recommendation) >= k and k != -1:
                break
        return recommendation

# Results
sum(rule.s * rule.c) --> 23.020

In [10]:
ur = User_Recommender(user_rules)
hr = hitrate(ur, 10)

100%|██████████| 101/101 [01:05<00:00,  1.54it/s]

avg len: 2.079208
HR@10	0.18069


In [11]:
class Combo_Recommender(Recommender):
    def __init__(self, ar, ur):
        self.ar = ar
        self.ur = ur
        
    def recommend(self, user_items, k, user_id, split):
        recommendation = self.ur.recommend(user_items, k, user_id, split)
        if len(recommendation) < k:
            recommendation.update(self.ar.recommend(user_items, k, user_id, split))
        return recommendation

In [22]:
cr = Combo_Recommender(ar, ur)
hr = hitrate(cr, 10)

100%|██████████| 101/101 [02:59<00:00,  1.78s/it]

avg len: 10.000000
HR@10	0.32178
nDCG@10	0.07516


In [13]:
class Combo_Recommender2(Recommender):
    def __init__(self, ar, ur):
        self.ar = ar
        self.ur = ur
        
    def recommend(self, user_items, k, user_id, split):
        recommendation = self.ur.recommend(user_items, 8, user_id, split)
        if len(recommendation) < k:
            recommendation.update(self.ar.recommend(user_items, k - len(recommendation), user_id, split))
        return recommendation

In [21]:
cr = Combo_Recommender2(ar, ur)
hr = hitrate(cr, 10)

100%|██████████| 101/101 [03:01<00:00,  1.79s/it]

avg len: 9.997525
HR@10	0.30941
nDCG@10	0.06935
